In [1]:
from datasets import load_dataset

beer_dataset = load_dataset("lengocquangLAB/beer-com-sentences")
beer_dataset

README.md:   0%|          | 0.00/986 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/663k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/93.4k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/88.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1612 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1547 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['profileName', 'beerId', 'beerName', 'beerABV', 'beerStyle', 'reviewSentence', 'appearanceRate', 'aromaRate', 'palateRate', 'tasteRate', 'appearance', 'aroma', 'palate', 'taste'],
        num_rows: 12036
    })
    test: Dataset({
        features: ['profileName', 'beerId', 'beerName', 'beerABV', 'beerStyle', 'reviewSentence', 'appearanceRate', 'aromaRate', 'palateRate', 'tasteRate', 'appearance', 'aroma', 'palate', 'taste'],
        num_rows: 1612
    })
    validation: Dataset({
        features: ['profileName', 'beerId', 'beerName', 'beerABV', 'beerStyle', 'reviewSentence', 'appearanceRate', 'aromaRate', 'palateRate', 'tasteRate', 'appearance', 'aroma', 'palate', 'taste'],
        num_rows: 1547
    })
})

In [ ]:
import logging
import warnings
import random
from tqdm import tqdm
from collections import Counter

import time
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import os
import pandas as pd


import transformers
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import (
    BertTokenizer,
    BertModel,
    PretrainedConfig,
    PreTrainedModel,
    Trainer,
    TrainingArguments
)


from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_recall_fscore_support,
    accuracy_score,
    confusion_matrix
)

from huggingface_hub import HfApi, create_repo

warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.tokenization_utils_base")
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

torch.set_default_tensor_type(torch.FloatTensor)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

aspects = ['appearance', 'aroma', 'palate', 'taste']

def set_random_set(seed):
    transformers.set_seed(seed)  # For transformers
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)

set_random_set(2024)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vocab_size = tokenizer.vocab_size
bert = BertModel.from_pretrained('bert-base-uncased')
hf_token = ""
del tokenizer, bert # only download for cached
torch.cuda.empty_cache()

2025-09-17 13:34:17.116751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758116057.277204      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758116057.334200      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [3]:
class BeerReviewDataset(Dataset):
    def __init__(self, ids, attention_masks, labels):
        self.ids = ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx]
        }

In [4]:

class BERTModelConfig(PretrainedConfig):
    model_type = "bert_model"

    def __init__(self, num_classes=1, pos_weight=None, **kwargs):
        super().__init__(**kwargs)
        self.num_classes = num_classes
        self.pos_weight = pos_weight 

class BERTModel(PreTrainedModel):
    config_class = BERTModelConfig

    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(768, config.num_classes)

        if config.pos_weight is not None:
            pos_weight = torch.tensor(config.pos_weight, dtype=torch.float32)
            self.criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        else:
            self.criterion = nn.BCEWithLogitsLoss()
        

    def forward(self, input_ids=None, attention_mask=None, labels=None):      
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        loss = None
        if labels is not None:
            loss = self.criterion(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)

In [5]:

def train_and_eval_model(aspect, model, method, train_dataset, val_dataset, test_dataset, batch_size, num_epochs, lr, weight_decay=0.01, patience=3):
    model_id = f"trungpq/slac-new-{aspect}-{method}"
    try:
        repo_info = HfApi().repo_info(repo_id=model_id, token=hf_token)
    except Exception as e:
        if "404" in str(e):  # If repo does not exist
            create_repo(model_id, token=hf_token)
        else:
            print(f"An error occurred: {e}")
            
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        labels = labels.flatten()
    
        # Vì num_classes=1 → output shape = [batch, 1]
        probs = torch.sigmoid(torch.tensor(logits)).numpy().flatten()
        preds = (probs >= 0.5).astype(int)
        # if val_null_count > 0:
        #     null_preds = np.full(val_null_count, 3)
        #     null_labels = np.full(val_null_count, 3)
        #     preds = np.concatenate([preds, null_preds])
        #     labels = np.concatenate([labels, null_labels])
    
        # --- macro ---
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
            labels, preds, average="macro"
        )
        # --- micro ---
        precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
            labels, preds, average="micro"
        )
    
        cm = confusion_matrix(labels, preds)
    
        tp = np.diag(cm)
        fp = cm.sum(axis=0) - tp
        fn = cm.sum(axis=1) - tp
        tn = cm.sum() - (tp + fp + fn)
    
        total_tp = tp.sum()
        total_fp = fp.sum()
        total_fn = fn.sum()
        total_tn = tn.sum()
    
        return {
            "accuracy": accuracy_score(labels, preds),
            "f1_macro": f1_macro,
            "precision_macro": precision_macro,
            "recall_macro": recall_macro,
            "f1_micro": f1_micro,
            "precision_micro": precision_micro,
            "recall_micro": recall_micro,
            "total_tf": [total_tp, total_fp, total_tn, total_fn],
        }


    num_warmup_steps = int(len(train_dataset) / batch_size)
    num_logging_steps = int(num_warmup_steps / 5)
    if num_logging_steps == 0:
        num_logging_steps = 1

    training_args = TrainingArguments(
        output_dir=f"./sentences/results/{aspect}/",
        eval_strategy="epoch",   
        save_strategy="epoch",
        learning_rate=lr,
        max_grad_norm=1.0,
        warmup_steps=num_warmup_steps,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=weight_decay,
        load_best_model_at_end=True,  # For early stopping
        metric_for_best_model="eval_f1_macro",
        greater_is_better=True,
        save_total_limit=1,  # Limit to save the best model only
        logging_dir='./logs',
        logging_steps=num_logging_steps,
        save_steps=10,
        report_to="none",
        disable_tqdm=False,
        push_to_hub=False,
        hub_model_id=model_id,
        hub_token=hf_token
    )

    # Initialize Trainer with model, arguments, train and evaluation datasets, and metrics
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    # Train the model
    trainer.train()
    trainer.push_to_hub()
    
    # tạo folder nếu chưa có
    os.makedirs("phase1_log", exist_ok=True)
    
    # lưu history ra csv
    history = pd.DataFrame(trainer.state.log_history)
    history.to_csv(f"phase1_log/{aspect}.csv", index=False)

    torch.cuda.empty_cache()
    time.sleep(10)
    trainer = Trainer(
        model=BERTModel.from_pretrained(model_id).to(device),
        args=training_args,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Evaluate the model
    torch.cuda.empty_cache()
    eval_results = trainer.evaluate()
    print(f"Test results: {eval_results}")
    
    # --- bổ sung labels/preds ---
    logits, labels = trainer.predict(test_dataset)[:2]
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs >= 0.5).float().numpy()
    eval_results["eval_labels"] = labels.tolist()
    eval_results["eval_preds"] = preds.tolist()
    
    del trainer

    return eval_results

In [6]:
def _truncate_seq_pair(tokens_a, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    while True:
        total_length = len(tokens_a)
        if total_length <= max_length:
            break
        tokens_a.pop()
     
def get_dataset(aspect, method, tokenizer, set_name, device='cuda', upsampling_time=5):
    df = beer_dataset[set_name].to_pandas()
    df = df[['reviewSentence', aspect]]

    df.dropna(inplace=True)
    # df = df.sample(n=100)

    sent = df['reviewSentence'].astype(str).to_list()
    labels = torch.tensor(df[aspect].values, dtype=torch.long).unsqueeze(1) 
    
    if method == 'upsample_replacement' and set_name == 'train':
        class_0_indices = [i for i, label in enumerate(labels) if label == 0]
        class_1_indices = [i for i, label in enumerate(labels) if label == 1]

        num_max_class = max(len(class_0_indices), len(class_1_indices))
        resampled_indices = []
        for class_indices in [class_0_indices, class_1_indices]:
            if len(class_indices) < num_max_class:
                num_upsampling = num_max_class
                resampled_indices.append(np.random.choice(class_indices, num_upsampling, replace=True))
            else:
                resampled_indices.append(class_indices)

        upsampled_indices = np.concatenate(resampled_indices)
        np.random.shuffle(upsampled_indices)

        sent = [sent[i] for i in upsampled_indices]
        labels = labels[upsampled_indices]

    sent = [s.replace("\n", ",").replace(".", " ") for s in sent]

    max_seq_length = 128
    total_input_ids = []
    total_input_mask = []

    for s in tqdm(sent):
        tokens = tokenizer.tokenize(s)

        _truncate_seq_pair(tokens, max_seq_length - 3) #account for [CLS] [SEP] [SEP]

        tokens = ["[CLS]"] + tokens + ["[SEP]"]

        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)

        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length

        total_input_ids.append(torch.tensor([input_ids], dtype=torch.long).to(device))
        total_input_mask.append(torch.tensor([input_mask], dtype=torch.long).to(device))

    assert len(total_input_ids) == len(sent)
    assert len(total_input_mask) == len(sent)

    input_ids = torch.cat(total_input_ids, dim=0).to('cpu')
    attention_masks = torch.cat(total_input_mask, dim=0).to('cpu')

    labels = torch.tensor(labels, dtype=torch.float)

    return input_ids, attention_masks, labels

In [7]:
def compute_class_weights(labels):
    flattened_labels = [int(label.item()) for label in labels]

    class_counts = Counter(flattened_labels)
    total_count = sum(class_counts.values())
    pos_count = class_counts[1] if 1 in class_counts else 0
    neg_count = class_counts[0] if 0 in class_counts else 0
    pos_weight = neg_count / pos_count
    
    return float(pos_weight)

def get_datataset_and_model(method, aspect):

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    train_data = get_dataset(aspect, method, tokenizer, 'train')
    val_data = get_dataset(aspect, method, tokenizer, 'validation')
    test_data = get_dataset(aspect, method, tokenizer, 'test')

    train_dataset = BeerReviewDataset(*train_data)
    
    val_dataset = BeerReviewDataset(*val_data)

    test_dataset = BeerReviewDataset(*test_data)

    del tokenizer
    torch.cuda.empty_cache()

    if method == "class_weight":
        pos_weight = compute_class_weights([item['labels'] for item in train_dataset])
    else:
        pos_weight = None
    
    return train_dataset, val_dataset, test_dataset, pos_weight

In [8]:
def train(method, model_name):
    f1_macro = 0
    precision_macro = 0
    recall_macro = 0
    TFs = (0, 0, 0, 0)
    
    # gom labels/preds toàn bộ aspects để tính macro-class
    all_labels, all_preds = [], []
    
    for aspect in aspects:
        print(f'------ Aspect: {aspect}------')
        train_dataset, val_dataset, test_dataset, pos_weight = get_datataset_and_model(method, aspect)

        # if model_name == "LSTM":
        #     model = LSTMModel(num_classes=1, criterion=criterion).to(device)
        #     num_epochs=8
        # elif model_name == "CNN":
        #     model = CNNModel(num_classes=1, criterion=criterion).to(device)
        #     num_epochs=8
        # else:
        model = BERTModel(BERTModelConfig(num_classes=1, pos_weight=pos_weight)).to(device)
        num_epochs=15

        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)

        batch_size=64
        lr = 2e-5
        weight_decay = 0.01

        result = train_and_eval_model(aspect, model, method, train_dataset, val_dataset, test_dataset, batch_size=batch_size, num_epochs=num_epochs, lr=lr, weight_decay=weight_decay, patience=5)

        f1_macro += result['eval_f1_macro']
        precision_macro += result['eval_precision_macro']
        recall_macro += result['eval_recall_macro']
        TFs = np.add(TFs, result['eval_total_tf']).tolist()
        
        # gom labels/preds để tính macro-class flatten
        all_labels.extend(result["eval_labels"])
        all_preds.extend(result["eval_preds"])

        del model, train_dataset, val_dataset

        torch.cuda.empty_cache()

    # --- Macro theo aspect (average đều các aspect) ---
    num_aspects = len(aspects)
    f1_macro /= num_aspects
    precision_macro /= num_aspects
    recall_macro /= num_aspects

    # --- Macro theo class (flatten all aspects) ---
    from sklearn.metrics import precision_recall_fscore_support
    p_macro_class, r_macro_class, f1_macro_class, _ = precision_recall_fscore_support(
        all_labels, all_preds, average="macro", zero_division=0
    )

    total_tp, total_fp, total_tn, total_fn = TFs
    
    # --- Micro (global TP/FP/FN) ---
    precision_micro = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall_micro = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_micro = (2 * precision_micro * recall_micro) / (precision_micro + recall_micro) if (precision_micro + recall_micro) > 0 else 0
    torch.cuda.empty_cache()
    # Trả về kết quả
    return {
        # macro-aspect
        "f1_macro": f1_macro,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        # macro-class (flatten)
        "f1_macro_class": f1_macro_class,
        "precision_macro_class": p_macro_class,
        "recall_macro_class": r_macro_class,
        # micro
        "f1_micro": f1_micro,
        "precision_micro": precision_micro,
        "recall_micro": recall_micro,
    }

In [9]:
for method in ["upsample_replacement", "class_weight"]:
    result = train(method, "BERT")
    
    eval_metrics = {
        'model': "BERT",
        'method': method,
        
        # --- macro ---
        'f1_macro': result['f1_macro'],
        'precision_macro': result['precision_macro'],
        'recall_macro': result['recall_macro'],
        
        'f1_macro_class': result['f1_macro_class'],
        'precision_macro_class': result['precision_macro_class'],
        'recall_macro_class': result['recall_macro_class'],
        
        # --- micro ---
        'f1_micro': result['f1_micro'],
        'precision_micro': result['precision_micro'],
        'recall_micro': result['recall_micro']
    }
    
    print("FINAL: ", eval_metrics)
    
    torch.cuda.empty_cache()

------ Aspect: appearance------


100%|██████████| 1612/1612 [00:00<00:00, 3101.87it/s]


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro,F1 Micro,Precision Micro,Recall Micro,Total Tf
1,0.106300,0.124008,0.961862,0.935410,0.916842,0.957292,0.961862,0.961862,0.961862,"[1488, 59, 1488, 59]"
2,0.027300,0.117256,0.974144,0.954464,0.950444,0.958608,0.974144,0.974144,0.974144,"[1507, 40, 1507, 40]"
3,0.020900,0.123847,0.973497,0.952830,0.953521,0.952142,0.973497,0.973497,0.973497,"[1506, 41, 1506, 41]"
4,0.015300,0.141364,0.971558,0.949301,0.950680,0.947936,0.971558,0.971558,0.971558,"[1503, 44, 1503, 44]"
5,0.003400,0.174756,0.974144,0.953338,0.960586,0.946454,0.974144,0.974144,0.974144,"[1507, 40, 1507, 40]"
6,0.014400,0.168617,0.970911,0.948228,0.948912,0.947547,0.970911,0.970911,0.970911,"[1502, 45, 1502, 45]"
7,0.003300,0.207215,0.967679,0.942032,0.946187,0.938005,0.967679,0.967679,0.967679,"[1497, 50, 1497, 50]"
8,0.006500,0.179176,0.974144,0.953482,0.959223,0.947973,0.974144,0.974144,0.974144,"[1507, 40, 1507, 40]"
9,0.002400,0.188932,0.974144,0.954050,0.954050,0.954050,0.974144,0.974144,0.974144,"[1507, 40, 1507, 40]"
10,0.003100,0.204676,0.972851,0.952187,0.948187,0.956310,0.972851,0.972851,0.972851,"[1505, 42, 1505, 42]"


Uploading...:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro,F1 Micro,Precision Micro,Recall Micro,Total Tf
1,0.419700,0.426016,0.891403,0.864531,0.847396,0.889904,0.891403,0.891403,0.891403,"[1379, 168, 1379, 168]"
2,0.334300,0.391771,0.907563,0.883247,0.867797,0.904113,0.907563,0.907563,0.907563,"[1404, 143, 1404, 143]"
3,0.290100,0.461752,0.887524,0.861460,0.842445,0.892435,0.887524,0.887524,0.887524,"[1373, 174, 1373, 174]"
4,0.232800,0.575660,0.907563,0.880328,0.871584,0.890456,0.907563,0.907563,0.907563,"[1404, 143, 1404, 143]"
5,0.146500,0.602433,0.910149,0.882733,0.876494,0.889622,0.910149,0.910149,0.910149,"[1408, 139, 1408, 139]"
6,0.146800,0.750719,0.910149,0.882541,0.876847,0.888769,0.910149,0.910149,0.910149,"[1408, 139, 1408, 139]"
7,0.086100,0.716221,0.899160,0.871375,0.858771,0.887403,0.899160,0.899160,0.899160,"[1391, 156, 1391, 156]"
8,0.048200,0.993072,0.913381,0.883343,0.888969,0.878124,0.913381,0.913381,0.913381,"[1413, 134, 1413, 134]"
9,0.042600,1.007408,0.913381,0.884953,0.884953,0.884953,0.913381,0.913381,0.913381,"[1413, 134, 1413, 134]"
10,0.025100,1.053666,0.908209,0.879311,0.875631,0.883205,0.908209,0.908209,0.908209,"[1405, 142, 1405, 142]"


Uploading...:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Test results: {'eval_loss': 1.0219700336456299, 'eval_model_preparation_time': 0.0026, 'eval_accuracy': 0.9094292803970223, 'eval_f1_macro': 0.8802236010683344, 'eval_precision_macro': 0.8894126178390138, 'eval_recall_macro': 0.8721295153132342, 'eval_f1_micro': 0.9094292803970223, 'eval_precision_micro': 0.9094292803970223, 'eval_recall_micro': 0.9094292803970223, 'eval_total_tf': [1466, 146, 1466, 146], 'eval_runtime': 5.5939, 'eval_samples_per_second': 288.168, 'eval_steps_per_second': 4.648}
FINAL:  {'model': 'BERT', 'method': 'class_weight', 'f1_macro': 0.9076565081046444, 'precision_macro': 0.9156088618400844, 'recall_macro': 0.9004849658906104, 'f1_macro_class': 0.908722856091277, 'precision_macro_class': 0.9171933584876025, 'recall_macro_class': 0.9007875691442861, 'f1_micro': 0.9516129032258065, 'precision_micro': 0.9516129032258065, 'recall_micro': 0.9516129032258065}
